In [32]:
import pandas as pd
import numpy as np
import seaborn as sns

# 1. Analyse de la target

In [ ]:
data = pd.read_csv("../../data/airbnbprediction.csv")
data

In [ ]:
""" Utiliser la bibliothèque pandas pour récupérer les données, vous
supprimerez ensuite les 15 features suivantes :
'id','description', 'first_review', 'host_has_profile_pic', 'host_identity_verified',
'host_response_rate', 'host_since', 'last_review', 'latitude', 'longitude', 'name','number_of_reviews',
'review_scores_rating', 'thumbnail_url', 'zipcode'
"""

data = data.drop(['id','description', 'first_review', 'host_has_profile_pic', 'host_identity_verified', 'host_response_rate', 'host_since', 'last_review', 'latitude', 'longitude', 'name','number_of_reviews', 'review_scores_rating', 'thumbnail_url', 'zipcode'], axis=1)
data

In [ ]:
#Transformer la colonne log_price en une colonne price
data['price'] = np.exp(data['log_price'])
data

In [ ]:
# Identifiez les locations pour lesquelles il n’y a pas d’information sur les commodités (’{}’). Supprimez les features correspondantes
data = data.drop(data[data['amenities'] == '{}'].index)
data

In [ ]:
# Combien y a-t-il de doublons, supprimez ces doublons.
print("Nombre de doublons : ", data.duplicated().sum())
data = data.drop_duplicates()
data

In [ ]:
# Conserver maintenant dans la base de données seulement les locations comprises entre 20 et 1000 dollars, combien reste-t-il de samples.
data = data.drop(data[data['price'] < 20].index)
data = data.drop(data[data['price'] > 1000].index)
data

In [ ]:
# Afficher la répartition des prix avec un découpage en 100 bacs.
sns.histplot(data['price'], bins=100)


In [ ]:
"""
Identifier les trois types de propriétés les plus importantes en nombre,
afficher via histplot ces trois types de propriétés. Dans les quatre
prochaines questions vous vous limiterez aux samples ayant un prix
inférieur à 500 dollars. Que peut-on voir.
"""
trois_importantes= (data['property_type']).value_counts().head(3)
data = data.drop(data[data['price'] > 500].index)
sns.barplot(trois_importantes)


In [ ]:
#Calculer maintenant la valeur moyenne de ces propriétés, que constate-t-on. Comment peut-on expliquer ces différences apparentes.
data.groupby('property_type')['price'].mean()

In [ ]:
"""Effectué maintenant un affichage avec trois hisplot sur des dataframes ne
contenant qu’un seul type de propriété. Vous y positionnerez l’option
stat=‘percent’. Que peut-on constater, est-ce que ce type d’affichage
permet de mieux expliquer les résultats précédents.
"""
data1 = data[data['property_type'] == 'Apartment']
data2 = data[data['property_type'] == 'House']
data3 = data[data['property_type'] == 'Condominium']
sns.histplot(data1['price'], stat='percent')
sns.histplot(data2['price'], stat='percent')
sns.histplot(data3['price'], stat='percent')

In [ ]:
# Que peut-on conclure si l’on donne trop d’importance sur la seule valeur des moyennes.
data.groupby('property_type')['price'].mean()

2. Traitement de instant_bookable

In [ ]:
# Créer une liste contenant le nom de toutes les features objets. Pour chacune de ces features affichez le nombre de valeurs différentes.
data.select_dtypes(include=['object']).nunique()


In [ ]:
#Combien y a-t-il de samples avec la valeur ‘t’ ou ‘f’ dans instant_bookable, quel est le prix moyen des locations de chacune de ces catégories.
print(data['instant_bookable'].value_counts())
print(data.groupby('instant_bookable')['price'].mean())
# Y-a-t-il des données manquantes.
print(data['instant_bookable'].isnull().sum())

In [ ]:
"""
Créer une dataFrame pour chacune des deux catégories grâce à la
fonction groupby, vous ne traiterez que les locations de moins de 500
dollars.
Info : Cette fonction retourne un objet que vous pouvez parcourir comme une liste composée de
tuple (valeur du groupe, dataframe). Vous pouvez également récupérer chacune des dataframes via la
méthode get_group(valeur du groupe).
"""
data1 = data.groupby('instant_bookable').get_group('f')
data2 = data.groupby('instant_bookable').get_group('t')
data1

In [ ]:
data2

In [ ]:
"""
Afficher la via un histogramme la répartition des prix de chacune de
ces catégories. Que constatez-vous, est-il judicieux de conserver cette
feature ?
Info : Vous utiliserez un affichage normalisé grâce à l’option stat de histplot.
"""
sns.histplot(data1['price'], stat='percent')
sns.histplot(data2['price'], stat='percent')

# Traitement de room_type

In [ ]:
"""
Afficher le nombre et les différentes moyennes des différents types de
chambre, vous n’afficherez que les locations à moins de 500 dollars.
Que constatez-vous 
"""
data = data.drop(data[data['price'] > 500].index)
data.groupby('room_type')['price'].mean()

In [ ]:
"""Au regard de ces premières informations est-il nécessaire de créer une
feature pour chaque type de chambre, ou une seule colonne suffit"""
data['room_type'].value_counts()

In [ ]:
# Affichez la variation des prix des locations avec la fonction histplot.
sns.histplot(data['price'], bins=100)

In [ ]:
"""
Si l’on souhaite conserver une seule colonne, peut-on les numéroter
dans un ordre aléatoire.
"""
data['room_type'] = data['room_type'].astype('category')
data['room_type'] = data['room_type'].cat.codes
data['room_type']
"""
Sachant que la plupart des algorithmes d’apprentissage utilisent la
norme euclidienne pour comparer les samples entre eux, quel serait la
meilleure façon de représenter ces différents types.
"""
data['room_type'] = data['room_type'].astype('category')
data['room_type'] = data['room_type'].cat.codes
data['room_type'] = data['room_type'].astype('category')
data['room_type']




# Analyse des property_type

In [ ]:
#Combien y a-t-il de type de propriétés différentes et quel est le prix moyen de ces locations. Vous afficherez le résultat par ordre croissant du nombre du nombre de chaque propriété. Info : La fonction aggregate() combinée aux agrégations groupby permet d’appliquer plusieurs opérations classiques aux dataframes ou à une partie des features
data.groupby('property_type')['price'].mean().sort_values()
# Pensez-vous qu’il soit nécessaire de conserver toutes ces catégories. ?
# Non

In [ ]:
# On souhaite regrouper tous les types de propriétés qui ont un prix moyen élevé > 200 dollars. Créer une liste correspondant à ces types de propriétés et transformez leur valeur en Timeshare
data1 = data.groupby('property_type')['price'].mean()
data1 = data1[data1 > 200]
data1 = data1.index
data.loc[data['property_type'].isin(data1), 'property_type'] = 'Timeshare'
data

In [ ]:
# Créer une liste contenant les types de propriétés avec moins de 20 valeurs, et supprimez-les
data1 = data['property_type'].value_counts()
data1 = data1[data1 < 20]
data1 = data1.index
data = data.drop(data[data['property_type'].isin(data1)].index)
data

# Analyse des bed_type

In [ ]:
# Afficher le nombre de locations par types de lits ainsi que le prix moyen. Quelle analyse peut-on faire de ce premier résultat.
data.groupby('bed_type')['price'].mean()


In [ ]:
# Calculer le prix moyen en fonction de ces deux features. Info : Vous utiliserez la fonction groupby sur les deux features property_type et bed_type
data.groupby(['property_type', 'bed_type'])['price'].mean()

In [ ]:
# Effectuer le même travail sur le nombre de locations sur la combinaison des deux features
data = data.groupby(['property_type', 'bed_type']).size().reset_index()
data = data.pivot(index='property_type', columns='bed_type', values=0)
data
# Que constatez-vous concernant en particulier les types Other et loft, que peut-on en conclure. Voyez-vous une feature que semble avoir un impact fort sur les prix des locations.
# Les types de lits ont un impact fort sur les prix des locations

# Peut-on numériser ces données dans n’importe quel ordre ou faut-il conserver une certaine hiérarchie entre les différents types de lits.
# Il faut conserver une certaine hiérarchie entre les différents types de lits.


In [ ]:
"""Modifier le dataframe obtenue pour en faire une lecture plus simple."""
data = data.fillna(0)
data

In [ ]:
# Effectuer le même travail sur le nombre de locations sur la combinaison des deux features.
data = data.groupby(['property_type', 'bed_type']).size().reset_index()
data = data.pivot(index='property_type', columns='bed_type', values=0)
data

In [ ]:
# Que constatez-vous concernant en particulier les types Other et loft, que peut-on en conclure. Voyez-vous une feature que semble avoir un impact fort sur les prix des locations.
data.groupby(['property_type', 'bed_type'])['price'].mean()

In [ ]:
# Peut-on numériser ces données dans n’importe quel ordre ou faut-il conserver une certaine hiérarchie entre les différents types de lits.
data['bed_type'] = data['bed_type'].astype('category')
data['bed_type'] = data['bed_type'].cat.codes
data['bed_type'] = data['bed_type'].astype('category')
data['bed_type']


# Traitement de accommodates